In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import seaborn as sns

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau

from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2,EfficientNetB0,EfficientNetB4,Xception

import warnings
import glob
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv("D:\\archive\\insat_3d_ds - Sheet.csv")

In [4]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [5]:
train_data = train_datagen.flow_from_dataframe(train,directory="D:\\archive\\insat3d_ir_cyclone_ds\\CYCLONE_DATASET_INFRARED",subset="training",
                                               x_col="img_name",y_col="label",target_size=(256,256),batch_size=16,class_mode='raw')

Found 136 validated image filenames.


In [6]:
for image_batch, labels_batch in train_data:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(16, 256, 256, 3)
(16,)


In [7]:
def build_model():
    base = tf.keras.applications.Xception(weights="imagenet", include_top=False, input_tensor=Input(shape=(256,256, 3)))

    base.trainable = False

    flatten = base.output
    flatten = Flatten()(flatten)

    bboxHead = Dense(64, activation="relu")(flatten)
    bboxHead = Dense(16, activation="relu")(bboxHead)
    # bboxHead = Dense(1, activation="linear")(bboxHead)

    model = Model(inputs=base.input, outputs=bboxHead)
    
    return model

In [8]:
model = build_model()

In [ ]:
# from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout,Input
# from tensorflow.keras import regularizers
# from tensorflow.keras.models import Model
# inputs=Input(shape=(256,256,3))
# y=Conv2D(32,3,activation=None, padding="same",kernel_initializer="he_normal",kernel_regularizer=regularizers.L1L2(0.01))(inputs)
# y=tf.keras.layers.BatchNormalization()(y)
# y=tf.keras.activations.relu(y)
# y=Conv2D(6,3,activation=None, padding="same",kernel_initializer="he_normal",kernel_regularizer=regularizers.L1L2(0.01))(y)
# y=tf.keras.layers.BatchNormalization()(y)
# y=tf.keras.activations.relu(y)
# y=MaxPool2D()(y)

# y= Flatten()(y)
# outputs=Dense(1,activation='linear')(y)
# model_1=Model(inputs=inputs,outputs=outputs)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 127, 127, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [11]:
model.fit(train_data, epochs=10)

Epoch 1/10
9/9 [==============================] - 9s 617ms/step - loss: 37.3144 - root_mean_squared_error: 45.3290
Epoch 2/10
9/9 [==============================] - 6s 608ms/step - loss: 29.0909 - root_mean_squared_error: 38.1072
Epoch 3/10
9/9 [==============================] - 6s 612ms/step - loss: 25.2799 - root_mean_squared_error: 34.8484
Epoch 4/10
9/9 [==============================] - 6s 648ms/step - loss: 21.6236 - root_mean_squared_error: 31.3247
Epoch 5/10
9/9 [==============================] - 6s 622ms/step - loss: 19.4082 - root_mean_squared_error: 29.2985
Epoch 6/10
9/9 [==============================] - 6s 628ms/step - loss: 16.5012 - root_mean_squared_error: 25.9162
Epoch 7/10
9/9 [==============================] - 6s 632ms/step - loss: 14.9835 - root_mean_squared_error: 25.0328
Epoch 8/10
9/9 [==============================] - 6s 621ms/step - loss: 13.7778 - root_mean_squared_error: 24.2754
Epoch 9/10
9/9 [==============================] - 6s 652ms/step - loss: 12.5500 

In [12]:
def load_and_prep_image(filename, img_shape=256):
  
  img = tf.io.read_file(filename)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.resize(img, size = [img_shape, img_shape])
  img = img/255.
  return img

In [13]:
def pred_and_plot(model, filename):
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))
  
# Plot the image and predicted value
  plt.imshow(img)
  plt.title(f"Prediction: {pred}")
  plt.axis(False);

In [ ]:
pred_and_plot(model,"D:\\archive\\insat3d_ir_cyclone_ds\\CYCLONE_DATASET_INFRARED\\37.jpg")